Dataset: https://www.kaggle.com/datasets/grassknoted/asl-alphabet

In [ ]:
# imports
import os
import kaggle


In [ ]:
# dataset path
dataset_path = 'tmp/asl-alphabet'


In [ ]:
# download dataset
if not os.path.exists(dataset_path):
    parent_dir = os.path.dirname(dataset_path)

    os.makedirs(parent_dir, exist_ok=True)

    kaggle.api.authenticate()
    kaggle.api.dataset_download_files('grassknoted/asl-alphabet', path=parent_dir, unzip=True)


In [ ]:
import torch
from torchvision import transforms
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import torchvision
from torch import nn
from torchsummary import summary
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models
from torch.utils.data import DataLoader, random_split


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import random
import os


In [ ]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


In [ ]:
# Paths to your training and testing data
data_dir = '/content/data_dir/asl_alphabet_train/asl_alphabet_train'
Val_dir = '/content/data_dir/asl_alphabet_train/asl_alphabet_train'


In [ ]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [ ]:
# Load the dataset using ImageFolder
dataset = datasets.ImageFolder(root=data_dir, transform=transform)


In [ ]:
# Get class names
class_names = dataset.classes
num_classes = len(class_names)
class_names


In [ ]:
# Define the ratio for train/test split
train_ratio = 0.8
train_size = int(train_ratio * len(dataset))
test_size = len(dataset) - train_size


In [ ]:
# Split the dataset into training and testing sets
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])


In [ ]:
# Get the number of samples in each set
num_train_samples = len(train_dataset)
num_test_samples = len(test_dataset)

print(f'Number of training samples: {num_train_samples}')
print(f'Number of testing samples: {num_test_samples}')


In [ ]:
num_classes = len(dataset.classes)
num_classes


In [ ]:
# Create DataLoaders for training and testing sets
# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)


In [ ]:
# Example of iterating over the training data
for images, labels in train_loader:
    # Your training code here
    print(images.shape, labels.shape)
    break

# Example of iterating over the testing data
for images, labels in test_loader:
    # Your testing code here
    print(images.shape, labels.shape)
    break


In [ ]:
# Function to display random samples
def show_random_samples(dataset, class_names, num_samples=5):
    fig, axes = plt.subplots(1, num_samples, figsize=(15, 15))
    for i in range(num_samples):
        random_idx = random.randint(0, len(dataset) - 1)
        img, label = dataset[random_idx]
        img = img.permute(1, 2, 0)  # Convert from (C, H, W) to (H, W, C) for displaying
        axes[i].imshow(img)
        axes[i].set_title(class_names[label])
        axes[i].axis('off')
    plt.show()


In [ ]:
# Display random samples from the training set
show_random_samples(train_dataset, class_names)


In [ ]:
# Display random samples from the testing set
show_random_samples(test_dataset, class_names)


# My Simple Model

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=29):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.conv3 = nn.Conv2d(64, 128, 3, 1)
        self.fc1 = nn.Linear(128 * 23 * 23, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
        self.pool = nn.MaxPool2d(2, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 23 * 23)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [ ]:
# Instantiate and move the model to the GPU
my_model = SimpleCNN(num_classes=num_classes).to(device)
# Define loss function and optimizer
my_model_criterion = nn.CrossEntropyLoss()
my_model_optimizer = optim.Adam(my_model.parameters(), lr=0.001)


In [ ]:
summary(my_model, input_size=(3, 200, 200))


# AlexNet Model

In [ ]:
AlexNet_model = models.alexnet(pretrained=True)

# Freeze model parameters
for param in AlexNet_model.parameters():
    param.requires_grad = False


In [ ]:
# Modify the last fully connected layer to fit your number of classes
AlexNet_model.classifier[6] = nn.Linear(4096, num_classes)

# Move the model to the device (GPU if available)
AlexNet_model.to(device)

# Define loss function and optimizer
AlexNet_model_criterion = nn.CrossEntropyLoss()
AlexNet_model_optimizer = optim.Adam(AlexNet_model.parameters(), lr=0.001)


In [ ]:
summary(AlexNet_model, input_size=(3, 200, 200))


# Vgg16 Model

In [ ]:
# Load pre-trained VGG16 model
Vgg16_model = models.vgg16(pretrained=True)

# Freeze model parameters
for param in Vgg16_model.parameters():
    param.requires_grad = False

# Modify the last fully connected layer to fit your number of classes
Vgg16_model.classifier[6] = nn.Linear(Vgg16_model.classifier[6].in_features, num_classes)

# Move the model to the device (GPU if available)
Vgg16_model.to(device)

# Define loss function and optimizer
Vgg16_model_criterion = nn.CrossEntropyLoss()
Vgg16_model_optimizer = optim.Adam(Vgg16_model.parameters(), lr=0.001)


In [ ]:
summary(Vgg16_model, input_size=(3, 200, 200))


# Vgg19 Model

In [ ]:
# Load pre-trained VGG19 model
Vgg19_model = models.vgg19(pretrained=True)

# Freeze model parameters
for param in Vgg19_model.parameters():
    param.requires_grad = False

# Modify the last fully connected layer to fit your number of classes
num_classes = len(dataset.classes)
Vgg19_model.classifier[6] = nn.Linear(Vgg19_model.classifier[6].in_features, num_classes)

# Move the model to the device (GPU if available)
Vgg19_model.to(device)

# Define loss function and optimizer
Vgg19_model_criterion = nn.CrossEntropyLoss()
Vgg19_model_optimizer = optim.Adam(Vgg19_model.parameters(), lr=0.001)


In [ ]:
summary(Vgg19_model, input_size=(3, 200, 200))


# GoogleNet Model

In [ ]:
# Load pre-trained GoogleNet model
googlenet_model = models.googlenet(pretrained=True)

# Freeze model parameters
for param in googlenet_model.parameters():
    param.requires_grad = False

# Modify the last fully connected layer to fit your number of classes
num_classes = len(dataset.classes)
#googlenet_model.fc = nn.Linear(googlenet_model.fc[6].in_features, num_classes)
googlenet_model.fc = nn.Linear(googlenet_model.fc.in_features, num_classes)

# Move the model to the device (GPU if available)
googlenet_model.to(device)

# Define loss function and optimizer
googlenet_model_criterion = nn.CrossEntropyLoss()
googlenet_model_optimizer = optim.Adam(googlenet_model.parameters(), lr=0.001)


In [ ]:
summary(googlenet_model, input_size=(3, 200, 200))


# ResNet Model

In [ ]:
# Load pre-trained Resnet model
resnet18_model = models.resnet18(pretrained=True)

# Freeze model parameters
for param in resnet18_model.parameters():
    param.requires_grad = False

# Modify the last fully connected layer to fit your number of classes
num_classes = len(dataset.classes)
#googlenet_model.fc = nn.Linear(googlenet_model.fc[6].in_features, num_classes)
resnet18_model.fc = nn.Linear(resnet18_model.fc.in_features, num_classes)

# Move the model to the device (GPU if available)
resnet18_model.to(device)

# Define loss function and optimizer
resnet18_model_criterion = nn.CrossEntropyLoss()
resnet18_model_optimizer = optim.Adam(resnet18_model.parameters(), lr=0.001)


In [ ]:
summary(resnet18_model, input_size=(3, 200, 200))


# Train

In [ ]:
# Training loop
def train_model(model, train_loader, criterion, optimizer, device, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() * inputs.size(0)
        epoch_loss /= len(train_loader.dataset)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")
    return epoch_loss


## My Model Training

In [ ]:
# Train the model
my_model_loss=train_model(my_model, train_loader, my_model_criterion, my_model_optimizer, device, num_epochs=10)


In [ ]:
torch.save(my_model, 'ASL_my_model_V1.0.1.pth')


## AlexNet Model Training

In [ ]:
AlexNet_model_loss=train_model(AlexNet_model, train_loader, AlexNet_model_criterion, AlexNet_model_optimizer, device, num_epochs=10)


In [ ]:
torch.save(AlexNet_model, 'ASL_AlexNet_model_V1.0.1.pth')


## Vgg16 Model Training

In [ ]:
Vgg16_model_loss=train_model(Vgg16_model, train_loader, Vgg16_model_criterion, Vgg16_model_optimizer, device, num_epochs=10)


In [ ]:
torch.save(Vgg16_model, 'ASL_Vgg16_model_V1.0.1.pth')


## Vgg19 Model Training

In [ ]:
Vgg19_model_loss=train_model(Vgg19_model, train_loader, Vgg19_model_criterion, Vgg19_model_optimizer, device, num_epochs=10)


In [ ]:
torch.save(Vgg19_model, 'ASL_Vgg19_model_V1.0.1.pth')


## GoogleNet Model Training

In [ ]:
googlenet_model_loss=train_model(googlenet_model, train_loader, googlenet_model_criterion, googlenet_model_optimizer, device, num_epochs=10)


In [ ]:
torch.save(googlenet_model, 'ASL_googlenet_model_V1.0.1.pth')


## ResNet Model Training

In [ ]:
resnet18_model_loss=train_model(resnet18_model, train_loader, resnet18_model_criterion, resnet18_model_optimizer, device, num_epochs=10)


In [ ]:
torch.save(resnet18_model, 'ASL_resnet18_model_V1.0.1.pth')


In [ ]:
model_losses = {
    "My Model": my_model_loss,
    "AlexNet": AlexNet_model_loss,
    "Vgg16": Vgg16_model_loss,
    "Vgg19": Vgg19_model_loss,
    "GoogLeNet": googlenet_model_loss,
    "ResNet18": resnet18_model_loss
}


In [ ]:
df = pd.DataFrame.from_dict(model_losses, orient='index', columns=['Loss'])


In [ ]:
print(df.to_string())


# Evaluation

In [ ]:
def test_model(model, test_loader, criterion, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total * 100
    print(f"Test Accuracy: {accuracy:.2f}%")
    return accuracy


In [ ]:
# Test the model
my_model_accuracy=test_model(my_model, test_loader, my_model_criterion, device)


In [ ]:
# Test the model
AlexNet_model_accuracy=test_model(AlexNet_model, test_loader, AlexNet_model_criterion, device)


In [ ]:
# Test the model
Vgg16_model_accuracy=test_model(Vgg16_model, test_loader, Vgg16_model_criterion, device)


In [ ]:
# Test the model
googlenet_model_accuracy=test_model(googlenet_model, test_loader, googlenet_model_criterion, device)


In [ ]:
# Test the model
resnet18_model_accuracy=test_model(resnet18_model, test_loader, resnet18_model_criterion, device)


In [ ]:
# Create a dictionary to store results
model_results = {
    "Model": ["My Model", "AlexNet", "Vgg16", "Vgg19", "GoogLeNet", "ResNet18"],
    "Loss": [my_model_loss, AlexNet_model_loss, Vgg16_model_loss, Vgg19_model_loss, googlenet_model_loss, resnet18_model_loss],
    "Accuracy": [my_model_accuracy, AlexNet_model_accuracy, Vgg16_model_accuracy, Vgg19_model_accuracy, googlenet_model_accuracy, resnet18_model_accuracy]
}

# Create a Pandas DataFrame from the dictionary
df = pd.DataFrame(model_results)

# Print the DataFrame as a table
print(df.to_string())


# Visualization

In [ ]:
# if you need load saved model use this
MyModel = SimpleCNN()  # Define your model
MyModel = torch.load('ASL_my_model_V1.0.1.pth')
MyModel = MyModel.to(device)


In [ ]:
def predict_and_display_image(model, image_path, transform, device):
    # Load the image
    image = Image.open(image_path).convert('RGB')

    # Preprocess the image
    image_tensor = transform(image).unsqueeze(0).to(device)

    # Set the model to evaluation mode
    model.eval()

    with torch.no_grad():
        # Make a prediction
        outputs = model(image_tensor)
        _, predicted = torch.max(outputs, 1)
        predicted_class = class_names[predicted.item()]

    # Display the image and prediction
    plt.imshow(image)
    plt.title(f"Predicted: {predicted_class}")
    plt.axis('off')
    plt.show()


In [ ]:
image_path = 'D:/IAAA/DeepLearning/Practice/Sixth\Datasets/asl_alphabet_test/G_test.jpg'
predict_and_display_image(my_model, image_path, transform, device)


In [ ]:
image_path = 'D:/IAAA/DeepLearning/Practice/Sixth\Datasets/asl_alphabet_test/G_test.jpg'
predict_and_display_image(AlexNet_model, image_path, transform, device)


In [ ]:
image_path = 'D:/IAAA/DeepLearning/Practice/Sixth\Datasets/asl_alphabet_test/G_test.jpg'
predict_and_display_image(Vgg16_model, image_path, transform, device)


In [ ]:
image_path = 'D:/IAAA/DeepLearning/Practice/Sixth\Datasets/asl_alphabet_test/G_test.jpg'
predict_and_display_image(Vgg19_model, image_path, transform, device)


In [ ]:
image_path = 'D:/IAAA/DeepLearning/Practice/Sixth\Datasets/asl_alphabet_test/G_test.jpg'
predict_and_display_image(googlenet_model, image_path, transform, device)


In [ ]:
image_path = 'D:/IAAA/DeepLearning/Practice/Sixth\Datasets/asl_alphabet_test/G_test.jpg'
predict_and_display_image(resnet18_model, image_path, transform, device)
